In [ ]:
!pip install tslearn

     |████████████████████████████████| 793 kB 15.1 MB/s 


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

/usr/local/lib/python3.7/dist-packages/tslearn/clustering/kmeans.py:17: UserWarning:

Scikit-learn <0.24 will be deprecated in a future release of tslearn



In [2]:
from google.colab import drive
drive.mount('/content/drive')
smart_meter = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IS project/3_imputed_dataset/new_impute_linear.csv', index_col='Time', parse_dates=True)
#weather = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IS project/3_imputed_dataset/impute_weather.csv', index_col='Time', parse_dates=True).drop('Unnamed: 0', axis=1)
smart_meter.bfill(inplace=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 20 Builidngs

In [3]:
def use_sample_cluster(n_rows, n_buildings):
  #n_rows, n_buildings = 1000, 20
  # Sample data
  sample = smart_meter.iloc[:n_rows,:n_buildings].copy()
  df_reshape = sample.T.values.reshape(len(sample.columns), -1, 1)
  print(f'n_building: {len(sample.columns)} \n n_rows: {n_rows} \n X_shape: {df_reshape.shape}')

  # Scaling
  X = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(df_reshape)

  # KShape Clustering
  for k_cluster in [2,3,4]:
    kshape = KShape(n_clusters=k_cluster, n_init=1, max_iter=1, random_state=42, verbose=True).fit(X)
    file_name = f'{n_buildings}cols_{n_rows}rows_{k_cluster}K.sav'
    path = '/content/drive/MyDrive/Colab Notebooks/IS project/model/'
    kshape.to_pickle(path + file_name)
    X_cluster = kshape.predict(X)
    
    # Plotting All Building Classified by Clusters
    plots=[]
    colors = ['orange', 'rgb(76,114,176)', 'green', 'red']
    for k in range(k_cluster):
      for col in sample.columns[X_cluster == k].values:
        # Add building lines of each cluster
        cluster_buildings = pd.DataFrame(sample[col]).copy()
        df_4hour = cluster_buildings.resample('4H').mean().rename({col: '4H_Avg'}, axis='columns') # Average energy consumption in every 4 Hours
        df_4hour_smooth = df_4hour.rolling(96).mean()                                             # Smoothing the line
        plots.append(go.Scatter(name=col, x=df_4hour_smooth.index, y=df_4hour_smooth['4H_Avg'], mode='lines',line=dict(color=colors[k])))
        
    fig = go.Figure(plots)
    fig.update_layout(yaxis_title='Energy Consumption (kWh)', 
                      title=f'{k_cluster} Clusters of {len(sample.columns)} Builidings by using {n_rows} rows')
    fig.show()

In [ ]:
use_sample_cluster(10000,20)

n_building: 20 
 n_rows: 10000 
 X_shape: (20, 10000, 1)
0.020 --> 0.019 --> 0.019 --> 


0.012 --> 0.012 --> 0.012 --> 


0.009 --> 0.009 --> 0.009 --> 


In [ ]:
use_sample_cluster(20000,20)

Output hidden; open in https://colab.research.google.com to view.

# 30000 rows, Colab collapsed

In [ ]:
use_sample_cluster(30000,20)

n_building: 20 
 n_rows: 30000 
 X_shape: (20, 30000, 1)


In [4]:
use_sample_cluster(35000,20)

n_building: 20 
 n_rows: 35000 
 X_shape: (20, 35000, 1)


ValueError: ignored

### Spared Code

In [ ]:
  # Plotting Each Cluster
  plt.figure(figsize=(10,10))
  for k in range(k_cluster):
      plt.subplot(k_cluster, 1, 1 + k)
      for xx in X[X_cluster == k]:
          plt.plot(xx.ravel(), "k-", alpha=.2)
      plt.plot(kshape.cluster_centers_[k].ravel(), "r-")
      plt.xlim(0, X.shape[1])
      plt.title(f'{k+1} of {k_cluster} Clusters of 20 Buildings by using {n_rows} rows \n Buildings name: {sample.columns[X_cluster == k].values}')
  plt.tight_layout()
  plt.show()